# Data Collection

Here, I obtain training data to train a model for obstacle avoidance that will classify images as blocked or free, where -
- Blocked: An obstacle is in the way such that if the rover goes forward it would collide or
- Free: It is safe to move forward as least slightly further without colliding. 

Note: This code will not remove the existing labeled data in the dataset directory. So if you need to discard existing images, do so before running the cells here.

Start with importing all the required libraries

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1

Because our neural network accepts 224x224 pixel images as input, I set the image size to minimize the filesize of the dataset. In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [2]:
camera = Camera.instance(width=224, height=224, capture_width=224, capture_height=224, capture_fps=5)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

Create directories to store the images captured. These directory names are also the labels.

In [3]:
blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


Create all the necessary buttons.

In [4]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

Create the functions needed to save the captured images as free or as blocked in the correct directions which reflect the labels.

In [5]:
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

Clicking the UI buttons will trigger the above functions that label the images.

In [6]:
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Display all the widgets together here so that I can see the images and buttons side by side to label.

In [7]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Save the dataset as a .zip file for easier transfer to other places for training as needed.

In [8]:
!zip -r -q dataset.zip dataset